# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [7]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2:
            self.rows, self.cols = args
            self.data = [[0 for _ in range(self.cols)] for _ in range(self.rows)]
        elif len(args) == 1:
            self.data = args[0]
            self.rows = len(self.data)
            self.cols = len(self.data[0])
            for row in self.data:
                if len(row) != self.cols:
                    raise ValueError("All rows must have the same number of columns")
        else:
            raise TypeError("Matrix initialization requires 2 or 1 argument")

    def __getitem__(self, index):
        if isinstance(index, tuple):
            row, col = index
            return self.data[row][col]
        else:
            return self.data[index]

    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            row, col = index
            self.data[row][col] = value
        else:
            self.data[index] = value

    def __eq__(self, other):
        if isinstance(other, Matrix):
            return self.data == other.data
        elif isinstance(other, list):
            if self.rows != len(other) or self.cols != len(other[0]):
                return False
            return self.data == other
        else:
            return False

    def __str__(self):
        return str(self.data)

    def shape(self):
        return (self.rows, self.cols)

    def transpose(self):
        return Matrix([[self.data[j][i] for j in range(self.rows)] for i in range(self.cols)])

    def row(self, n):
        return Matrix([self.data[n]])

    def column(self, n):
        return Matrix([[self.data[i][n]] for i in range(self.rows)])

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix([[self.data[i][j] for j in range(n_0, n_1 + 1)] for i in range(m_0, m_1 + 1)])

    def __getitem__(self, index):
        if isinstance(index, tuple):
            row, col = index
            if isinstance(row, slice) and isinstance(col, slice):
                return Matrix([[self.data[i][j] for j in range(col.start, col.stop, col.step)] for i in range(row.start, row.stop, row.step)])
            elif isinstance(row, slice):
                return Matrix([[self.data[i][col]] for i in range(row.start, row.stop, row.step)])
            elif isinstance(col, slice):
                return Matrix([[self.data[row][j] for j in range(col.start, col.stop, col.step)]])
            else:
                return self.data[row][col]
        else:
            return self.data[index]

    def constant(n, m, c):
        return Matrix([[c for _ in range(m)] for _ in range(n)])

    def zeros(n, m):
        return constant(n, m, 0.0)

    def ones(n, m):
        return constant(n, m, 1.0)

    def eye(n):
        return Matrix([[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)])

    def scalarmul(self, c):
        return Matrix([[c * self.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def add(self, N):
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrices must have the same dimensions for addition")
        return Matrix([[self.data[i][j] + N.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def sub(self, N):
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrices must have the same dimensions for subtraction")
        return Matrix([[self.data[i][j] - N.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def mat_mult(self, N):
        if self.cols != N.rows:
            raise ValueError("Number of columns in the first matrix must equal the number of rows in the second matrix for multiplication")
        result = [[0 for _ in range(N.cols)] for _ in range(self.rows)]
        for i in range(self.rows):
            for j in range(N.cols):
                for k in range(self.cols):
                    result[i][j] += self.data[i][k] * N.data[k][j]
        return Matrix(result)

    def element_mult(self, N):
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication")
        return Matrix([[self.data[i][j] * N.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def equals(self, N):
        return self.data == N.data

    def __mul__(self, other):
        if isinstance(other, Matrix):
            return self.mat_mult(other)
        elif isinstance(other, (int, float)):
            return self.scalarmul(other)
        else:
            raise TypeError("Unsupported operand type for *")

    def __rmul__(self, other):
        return self * other

    def __add__(self, other):
        if isinstance(other, Matrix):
            return self.add(other)
        else:
            raise TypeError("Unsupported operand type for +")

    def __sub__(self, other):
        if isinstance(other, Matrix):
            return self.sub(other)
        else:
            raise TypeError("Unsupported operand type for -")

    def __eq__(self, other):
        return self.equals(other)

    def __iadd__(self, other):
        self = self + other
        return self

    def __isub__(self, other):
        self = self - other
        return self

    def __imul__(self, other):
        self = self * other
        return self 

In [11]:
matrix1 = Matrix([[1,2],[3,4]])
matrix2 = Matrix([[5,6],[7,8]])

result = matrix1 + matrix2
print(result)

result = matrix1 - matrix2
print(result)

result = matrix1 * matrix2
print(result)

result = matrix1 * 2
print(result)

result = matrix1.transpose()
print(result)

[[6, 8], [10, 12]]
[[-4, -4], [-4, -4]]
[[19, 22], [43, 50]]
[[2, 4], [6, 8]]
[[1, 3], [2, 4]]


In [ ]:
Write a function make_deck that returns a list of all of the cards in a standard card deck. The return should be a list of tuples of pairs of suit and value. For example the 10 of Clubs would be ('Clubs', 10) and Queen of Hearts would be ('Hearts', 'Queen'). Recall that a deck has 52 cards, divided into 4 suits (Clubs, Diamonds, Hearts, and Spades), and that each suit has 13 cards: 2 to 10, Jack, Queen, King, and Ace. Summit your solution with Lab 5

In [5]:
def make_deck():
    suits = ['Clubs', 'Diamonds', 'Hearts', 'Spades']
    values = [str(i) for i in range(2, 11)] + ['Jack', 'Queen', 'King', 'Ace']

    deck = []
    for suit in suits:
        for value in values:
             deck.append((suit, value))

    return deck